In [1]:
# Reading PDFs
from langchain_community.document_loaders import PyPDFLoader
from langchain_unstructured import UnstructuredLoader
from langchain_core.documents import Document

# Embeddings
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_huggingface import HuggingFaceEmbeddings

# Chat Models
from huggingface_hub import hf_hub_download
#from llama_cpp import Llama
from langchain_community.chat_models import ChatLlamaCpp
#from langchain_huggingface import ChatHuggingFace, HuggingFacePipeline
from langchain_ollama import ChatOllama
#from langchain_community.llms import GPT4All

# Chat structure
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, END, StateGraph
from langgraph.graph.message import add_messages

# Typing
from typing import Optional, Callable, Dict, Any
from typing_extensions import Annotated, List, TypedDict
from pydantic import BaseModel, Field
from langchain.schema import HumanMessage, AIMessage, SystemMessage

%load_ext autoreload
%autoreload 2

### Load PDF

In [2]:
file_path = '../papers/examples/ponds1.pdf'

# Load as individual pages:
loader = PyPDFLoader(file_path)
pages = []
async for page in loader.alazy_load():
    pages.append(page)

In [3]:
# Load as chunks of parsed text
loader = UnstructuredLoader(
    file_path=file_path,
    strategy="hi_res"
)
chunks = []
for doc in loader.lazy_load():
    chunks.append(doc)

# Filter out metadata
filter_keys = ['category', 'page_number']
filtered_chunks = []

for d in chunks:
    mdata = {k: v for k,v in d.metadata.items() if k in filter_keys}
    fdoc = Document(page_content = d.page_content, metadata = mdata)
    filtered_chunks.append(fdoc)

INFO: pikepdf C++ to Python logger bridge initialized
INFO: Reading PDF for file: ../papers/examples/ponds1.pdf ...


In [4]:
references_page = -1
ref_keywords = ["references", "bibliography", "works cited"]
for chunk in chunks:
    if chunk.metadata['category'] == 'Title':
        title = chunk.page_content.lower()
        if any(k in title for k in ref_keywords):
            references_page = int(chunk.metadata['page_number'])
            break

pages = pages[:references_page]

In [5]:
pages[-1].page_content

'N. Kuczy´nska-Kippen et al.: Knowl. Managt. Aquatic Ecosyst. (2013)409, 08\nlayer of bottom sediments may have created an additional ecological niche for the inhabiting\norganisms. Among rotifers, bdelloids were found to dominate in each season and in each\nwater body, and contributed to the extremely high densities of rotifers reached in the autumn.\nThey are a class of the phylum Rotifera, having obligatory parthenogenetic reproduction and\nbeing able to withstand harsh periods through anhydrobiosis. They are typically a part of\nbenthos or periphyton; however, some bdelloid species (e.g. Rotaria) can be found occasion-\nally in the plankton (Ricci and Melone, 2000), which was the case in the present study. The\nmass domination of Rotaria occurred as the examined water bodies were at the level of drying\nup. This is consistent with bdelloid’s fantastic ecological adaptability, as they can easily shift\nfrom an active to an anhydrobiotic stage, which helps them to live in particularl

In [6]:
references_page

9

In [7]:
# Full code and examples: 
# https://python.langchain.com/docs/how_to/document_loader_pdf/#extracting-tables-and-other-structures

import fitz
import matplotlib.patches as patches
import matplotlib.pyplot as plt
from PIL import Image


def plot_pdf_with_boxes(pdf_page, segments):
    pix = pdf_page.get_pixmap()
    pil_image = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)

    fig, ax = plt.subplots(1, figsize=(10, 10))
    ax.imshow(pil_image)
    categories = set()
    category_to_color = {
        "Title": "orchid",
        "Image": "forestgreen",
        "Table": "tomato",
    }
    for segment in segments:
        points = segment["coordinates"]["points"]
        layout_width = segment["coordinates"]["layout_width"]
        layout_height = segment["coordinates"]["layout_height"]
        scaled_points = [
            (x * pix.width / layout_width, y * pix.height / layout_height)
            for x, y in points
        ]
        box_color = category_to_color.get(segment["category"], "deepskyblue")
        categories.add(segment["category"])
        rect = patches.Polygon(
            scaled_points, linewidth=1, edgecolor=box_color, facecolor="none"
        )
        ax.add_patch(rect)

    # Make legend
    legend_handles = [patches.Patch(color="deepskyblue", label="Text")]
    for category in ["Title", "Image", "Table"]:
        if category in categories:
            legend_handles.append(
                patches.Patch(color=category_to_color[category], label=category)
            )
    ax.axis("off")
    ax.legend(handles=legend_handles, loc="upper right")
    plt.tight_layout()
    plt.show()


def render_page(doc_list: list, page_number: int, print_text=True) -> None:
    pdf_page = fitz.open(file_path).load_page(page_number - 1)
    page_docs = [
        doc for doc in doc_list if doc.metadata.get("page_number") == page_number
    ]
    segments = [doc.metadata for doc in page_docs]
    plot_pdf_with_boxes(pdf_page, segments)
    if print_text:
        for doc in page_docs:
            print(f"{doc.page_content}\n")

In [8]:
render_page(chunks,9)

N. Kuczy´nska-Kippen et al.: Knowl. Managt. Aquatic Ecosyst. (2013) 409, 08

layer of bottom sediments may have created an additional ecological niche for the inhabiting organisms. Among rotifers, bdelloids were found to dominate in each season and in each water body, and contributed to the extremely high densities of rotifers reached in the autumn. They are a class of the phylum Rotifera, having obligatory parthenogenetic reproduction and being able to withstand harsh periods through anhydrobiosis. They are typically a part of benthos or periphyton; however, some bdelloid species (e.g. Rotaria) can be found occasion- ally in the plankton (Ricci and Melone, 2000), which was the case in the present study. The mass domination of Rotaria occurred as the examined water bodies were at the level of drying up. This is consistent with bdelloid’s fantastic ecological adaptability, as they can easily shift from an active to an anhydrobiotic stage, which helps them to live in particularly ephemer

### Embeddings

In [14]:
embeddings = HuggingFaceEmbeddings(
    model_name="intfloat/multilingual-e5-large-instruct",
    model_kwargs={'device': 'mps'},
    encode_kwargs={'normalize_embeddings': False}
)

page_store = InMemoryVectorStore(
    embedding=embeddings
)

_ = page_store.add_documents(documents=pages)


chunk_store = InMemoryVectorStore(
    embedding=embeddings
)

_ = chunk_store.add_documents(documents=filtered_chunks)

INFO: Load pretrained SentenceTransformer: intfloat/multilingual-e5-large-instruct


In [15]:
query = (
    "Data collected and reported for physical or chemical attributes of individually stuided ponds. "
    "Examples include but are not limited to depth, surface area, temperature, or pH."
)

In [16]:
# Retreive pages:
# NOTE: Depth seems to find the right page, but surface area does not. 
retreived = page_store.similarity_search(query = query, k = 5)
print(retreived[0].page_content)

N. Kuczy´nska-Kippen et al.: Knowl. Managt. Aquatic Ecosyst. (2013)409, 08
Table II
Descriptive data on zooplankton community structure and physical-chemical parameters in particular
seasons and in certain crater.
pond No.
 I
 I
 I
 II
 II
 II
 III
 III
 IV
 IV
Parameter
 spring
 summer
 autumn
 spring
 summer
 autumn
 spring
 summer
 spring
 summer
Rotifera
density
 188
 555
 10150
 30
 34
 64
 560
 220
 48
 324
(ind L−1)
Rotifera
number
 4
 13
 13
 5
 11
 10
 7
 4
 8
 13
of species
Cladocera
density
 3
 316
 60
 10
 120
 99
 25
 299
 3
 39
(ind L−1)
Cladocera
number
 3
 11
 8
 7
 9
 5
 7
 11
 3
 9
of species
Copepoda
density
 18
 23
 2
 12
 1
 1
 1
 3
 25
 3
(ind L−1)
Copepoda
number
 2
 3
 1
 4
 1
 1
 1
 1
 4
 2
of species
Rotifera
 0.17
 1.51
 0.46
 0.74
 1.94
 0.97
 0.57
 0.6
 0.96
 1.32
Shannon
Crustacea
 0.44
 1.59
 1.07
 1.87
 1.2
 0.28
 1.56
 1.07
 0.78
 1.69
Shannon
DIN (mg·L−1)
 9.197
 7.334
 7.695
 6.824
 8.481
 8.719
 6.534
 5.19
 19.287
 24.984
TP (mg·L−1)
 1.23
 1.41
 1.

In [17]:
# Retreive chunks:
retreived = chunk_store.similarity_search(query = query, k = 10)
print(retreived[0].page_content)

Descriptive data on zooplankton community structure and physical-chemical parameters in particular seasons and in certain crater.


### Load model

In [10]:
model_path = hf_hub_download(
    repo_id="google/gemma-3-12b-it-qat-q4_0-gguf",
    filename="gemma-3-12b-it-q4_0.gguf",
)

llm = ChatLlamaCpp(
    model_path=model_path,
    chat_format = "gemma",
    temperature=0,
    max_tokens=200,
    n_ctx=10000,
    stop = ["<end_of_turn>"]
)

simple_prompt_template = PromptTemplate.from_template(
    "<start_of_turn>user\n{message}<end_of_turn>\n"
    "<start_of_turn>model\n"
)

prompt_template = PromptTemplate.from_template(
    "<start_of_turn>user\n{instructions}<end_of_turn>\n"
    "<start_of_turn>user\n{context}<end_of_turn>\n"
    "<start_of_turn>user\n{query}<end_of_turn>\n"
    "<start_of_turn>model\n"
)

llama_model_load_from_file_impl: using device Metal (Apple M4) - 16383 MiB free
llama_model_loader: loaded meta data with 39 key-value pairs and 626 tensors from /Users/kquinn/.cache/huggingface/hub/models--google--gemma-3-12b-it-qat-q4_0-gguf/snapshots/7929e34ac7c459bbfd1f38d7358e6839f204578b/gemma-3-12b-it-q4_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = gemma3
llama_model_loader: - kv   1:                      gemma3.context_length u32              = 131072
llama_model_loader: - kv   2:                         gemma3.block_count u32              = 48
llama_model_loader: - kv   3:                    gemma3.embedding_length u32              = 3840
llama_model_loader: - kv   4:                 gemma3.feed_forward_length u32              = 15360
llama_model_loader: - kv   5:                gemma3.attention.head_

In [23]:
# Load the language model
llm = ChatOllama(
    model="gemma3:12b-it-qat",
    temperature=0
)

instructions = (
        "You will be given contextual information from a page of a scientific research paper "
        "and asked to accurately answer questions about its contents. Please answer only "
        "for the information shown on the current page, and not the paper as a whole."
        "If applicable, point to a specific location or excerpt "
        "from the text to justify your reasoning."
)

# Specify prompt and response formats
prompt_template = PromptTemplate.from_template(
    "<start_of_turn>user\n{instructions}<end_of_turn>\n"
    "<start_of_turn>user\n{context}<end_of_turn>\n"
    "<start_of_turn>user\n{query}<end_of_turn>\n"
    "<start_of_turn>model\n"
)

class StructuredResponse(BaseModel):
    """
    Manages a structured response from a language model.
    """
    response : bool = Field(
        description = 
            "'False' if the answer is No or Unknown. "
            "'True' only if the answer is Yes. "
    )
    
    reason : str = Field(
        description = 
            "Specify location or excerpt "
            "from the text to justify your reasoning. "
    )


llm = llm.with_structured_output(schema = StructuredResponse, method = "json_mode")

### Chat

In [11]:
from langchain_core.utils.function_calling import convert_to_openai_tool

class StructuredResponse(BaseModel):
    """
    Manages a structured response from a language model.
    """
    response : bool = Field(
        description = 
            "Respond with False for a negative or unknown answer. "
            "Respond True only if the question is answered affirmitavely. "
    )
    
    reason : str = Field(
        description = 
            "Point to a specific location or excerpt "
            "from the text to justify your reasoning. "
            "'None' if nothing extracted."
    )

dict_schema = convert_to_openai_tool(StructuredResponse)
llm = llm.with_structured_output(dict_schema, method = "json_mode")

ValueError: Received unsupported arguments {'method': 'json_mode'}

In [12]:
dict_schema

{'type': 'function',
 'function': {'name': 'StructuredResponse',
  'description': 'Manages a structured response from a language model.',
  'parameters': {'properties': {'response': {'description': 'Respond with False for a negative or unknown answer. Respond True only if the question is answered affirmitavely. ',
     'type': 'boolean'},
    'reason': {'description': "Point to a specific location or excerpt from the text to justify your reasoning. 'None' if nothing extracted.",
     'type': 'string'}},
   'required': ['response', 'reason'],
   'type': 'object'}}}

In [22]:
#llm = llm.with_structured_output(StructuredResponse)

instructions = (
        "You will be given contextual information from a scientific research paper "
        "and asked to accurately answer questions about its contents. "
        "If applicable, point to a specific location or excerpt "
        "from the text to justify your reasoning."
    )

In [ ]:
class State(TypedDict):
    retrieval_query : str
    prompt : str
    context : List[Document]
    answer : StructuredResponse


def retrieve(state: State):
    retrieved_docs = page_store.similarity_search(state["retrieval_query"], k = 2)
    return {"context": retrieved_docs}


def generate(state: State):
    # For full document:
    #docs_content = "\n\n".join(doc.page_content for doc in pages)

    # For retrieved text:
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    
    messages = prompt_template.invoke({"prompt": state["prompt"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response}


# Compile application and test
graph_builder = StateGraph(State)
graph_builder.add_node("generate", generate)
graph_builder.add_node("retrieve", retrieve)
graph_builder.add_edge(START, "retrieve")
graph_builder.add_edge("retrieve", "generate")
graph_builder.add_edge("generate", END)
graph = graph_builder.compile()

In [ ]:
retrieval_query = (
    "Data collected and reported for physical or chemical attributes of individually stuided ponds. "
    "Examples include but are not limited to depth, surface area, temperature, or pH."
)
prompt = (
    "Does the paper report data related to physical or chemical attributes of individual ponds? "
    "Examples include but are not limited to depth, surface area, temperature, or pH. "
    "Give reasoning and textual evidence to support your conclusion."
)
output = graph.invoke({"retrieval_query": retrieval_query, "prompt" : prompt})

ConnectError: [Errno 61] Connection refused

In [ ]:
retrieval_query = (
    "Scientific definitions for ponds specifying distinguishing physical or chemical attributes."
)
prompt = (
    "Does the paper contain a concrete, scientific definition for a pond? "
    "A definition should specify distinguishing physical or chemical attributes. "
    "Give reasoning or textual evidence to support your conclusion."
)
output = graph.invoke({"retrieval_query": retrieval_query, "prompt" : prompt})

INFO: HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


In [ ]:
instructions = (
        "You will be given contextual information from a page of a scientific research paper "
        "and asked to accurately answer questions about its contents. Please answer only "
        "for the information shown on the current page, and not the paper as a whole."
        "Your answer should be given in JSON format, with keys for 'response' and 'reason.' "
        "The 'response' key should be a boolean value, with a value of False if the "
        "answer is No or Unknown and a value of True only if the answer is Yes. "
        "The 'reason' key should contain a description of your reasoning, "
        "with a specific location or excerpt from the text for justification. "

)

context = pages[2].page_content

query1 = (
    "Does this page include any numerical data related to physical or chemical attributes of individual ponds? "
    "Examples may include quantitative measurements of depth, surface area, temperature, or pH. "
)

query2 = (
    "Does this page contain a scientific definition for classifying either ponds or lakes?"
)

prompt = prompt_template.invoke({
    "instructions": instructions,
    "context": context,
    "query": query1
})

answer = llm.invoke(prompt)
print(answer)

INFO: HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


response=True reason="Yes, the page includes numerical data related to physical attributes of individual ponds. Table I provides data on 'Maximum depth [m]' (1, 1, 0.3, 0.7), 'Diameter [m]' (40, 28, 30, 13), and 'Surface area [m2]' (1100, 517, 197, 135) for ponds I, II, III, and IV. Additionally, the text mentions that the maximum depth of the largest water body does not exceed 1m (Table I)."


In [20]:
answer

StructuredResponse(response='No, this page does not include any numerical data related to physical or chemical attributes of individual ponds. It mentions chlorophyll a concentration, pH, and conductivity as factors influencing zooplankton assemblages, but does not provide specific values for any of these parameters in any of the ponds.', reason='The text states, "seasonality, physical-chemical features (mainly chlorophyll a concentration, pH and conductivity) or biological parameters...structured zooplankton assemblages." However, it does not provide any numerical values for these features in the ponds.')

In [ ]:
# Papers
papers = {
    'Pond 1: Specificity of zooplankton distribution in meteorite craterponds (Morasko, Poland)' :
    '../papers/examples/ponds1.pdf',
    'Pond 2: Environmental Conditions and Macrophytes of Karst Ponds' :
    '../papers/examples/ponds2.pdf',
    'Pond 3: Drivers of carbon dioxide and methane supersaturation in small, temporary ponds' :
    '../papers/examples/ponds3.pdf',
    'Lake 1: Lake metabolism scales with lake morphometry and catchment conditions' :
    '../papers/examples/lakes1.pdf',
    'Lake 2: Net Heterotrophy in Small Danish Lakes: A Widespread Feature  Over Gradients in Trophic Status and Land Cover' :
    '../papers/examples/lakes2.pdf',
    'Lake 3: Patterns in the Species Composition and Richness of Fish Assemblages in Northern Wisconsin Lakes' :
    '../papers/examples/lakes3.pdf',
    'Definition 1: Eutrophication: are mayflies (Ephemeroptera) good bioindicators for ponds?' :
    '../papers/examples/definitions1.pdf',
    'Definition 2: The importance of small waterbodies for biodiversity and ecosystem services: implications for policy makers' :
    '../papers/examples/definitions2.pdf',
    'Definition 3: Agricultural Freshwater Pond Supports Diverse and Dynamic Bacterial and Viral Populations' :
    '../papers/examples/definitions3.pdf',
    'Fake 1: Quantifying saltmarsh vegetation and its effect on wave height dissipation: Results from a UK East coast saltmarsh' :
    '../papers/examples/fake1.pdf',
    'Fake 2: Methane and Carbon Dioxide Fluxes in a Temperate Tidal Salt Marsh: Comparisons Between Plot and Ecosystem Measurements' :
    '../papers/examples/fake2.pdf',
    'Fake 3: Improving the definition of a coastal habitat: Putting the salt back into saltmarsh' :
    '../papers/examples/fake3.pdf',
}
titles = list(papers.keys())

# Load the language model
model_path = hf_hub_download(
    repo_id="google/gemma-3-12b-it-qat-q4_0-gguf",
    filename="gemma-3-12b-it-q4_0.gguf",
)

llm = ChatLlamaCpp(
    model_path=model_path,
    chat_format = "gemma",
    temperature=0,
    max_tokens=1000,
    n_ctx=10000
)

# Specify prompt and response formats
prompt_template = PromptTemplate.from_template(
    "<start_of_turn>user\n{instructions}<end_of_turn>\n"
    "<start_of_turn>user\n{context}<end_of_turn>\n"
    "<start_of_turn>user\n{query}<end_of_turn>\n"
    "<start_of_turn>model\n"
)

class StructuredResponse(BaseModel):
    """
    Manages a structured response from a language model.
    """
    response : bool = Field(
        description = 
            "False if the answer is no or unknown. "
            "True only if the answer is yes. "
    )
    
    reason : str = Field(
        description = 
            "Specify location or excerpt "
            "from the text to justify your reasoning. "
    )

dict_schema = convert_to_openai_tool(StructuredResponse)
structured_llm = llm.with_structured_output(dict_schema)


# Build the conditional graph:
class State(TypedDict):
    title : str
    page_num : int
    context : Document
    screen : bool
    reason : str
    definition : str


def screen_for_definition(state : State):
    query = "Does this page contain a scientific definition for classifying either ponds or lakes?"
    messages = prompt_template.invoke(
        {"instructions": instructions, "context": state['context'], "query": query}
    )
    answer = structured_llm.invoke(messages)
    return {"screen": answer['response'], "reason": answer['reason']}


def extract_definition(state : State):
    query = "What is the definition for classifying either ponds or lakes given by the context?"
    messages = prompt_template.invoke(
        {"instructions": instructions, "context": state['context'], "query": query}
    )
    answer = llm.invoke(messages)
    return {"definition": answer.content}


def extract_routing(state : State):
    return state['screen']


graph_builder = StateGraph(State)
graph_builder.add_node("screen_for_definition", screen_for_definition)
graph_builder.add_node("extract_definition", extract_definition)
graph_builder.add_edge(START, "screen_for_definition")
graph_builder.add_conditional_edges(
    "screen_for_definition",
    extract_routing,
    {True : "extract_definition", False: END}
)
graph_builder.add_edge("extract_definition", END)
graph = graph_builder.compile()

llama_model_load_from_file_impl: using device Metal (Apple M4) - 16378 MiB free
llama_model_loader: loaded meta data with 39 key-value pairs and 626 tensors from /Users/kquinn/.cache/huggingface/hub/models--google--gemma-3-12b-it-qat-q4_0-gguf/snapshots/7929e34ac7c459bbfd1f38d7358e6839f204578b/gemma-3-12b-it-q4_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = gemma3
llama_model_loader: - kv   1:                      gemma3.context_length u32              = 131072
llama_model_loader: - kv   2:                         gemma3.block_count u32              = 48
llama_model_loader: - kv   3:                    gemma3.embedding_length u32              = 3840
llama_model_loader: - kv   4:                 gemma3.feed_forward_length u32              = 15360
llama_model_loader: - kv   5:                gemma3.attention.head_

In [58]:
graph

In [100]:
paper = list(papers.items())[0]
title = paper[0]
file_path = paper[1]

# Load as individual pages:
loader = PyPDFLoader(file_path)
pages = []
async for page in loader.alazy_load():
    pages.append(page)

page_num = 0

response = graph.invoke(
    {
        'title' : title,
        'page_num' : page_num,
        'context' : pages[0].page_content
    }
)

Llama.generate: 919 prefix-match hit, remaining 25 prompt tokens to eval
llama_perf_context_print:        load time =   53540.18 ms
llama_perf_context_print: prompt eval time =    2436.44 ms /    25 tokens (   97.46 ms per token,    10.26 tokens per second)
llama_perf_context_print:        eval time =    6883.73 ms /    61 runs   (  112.85 ms per token,     8.86 tokens per second)
llama_perf_context_print:       total time =   10796.69 ms /    86 tokens
Llama.generate: 919 prefix-match hit, remaining 26 prompt tokens to eval
llama_perf_context_print:        load time =   53540.18 ms
llama_perf_context_print: prompt eval time =    1381.39 ms /    26 tokens (   53.13 ms per token,    18.82 tokens per second)
llama_perf_context_print:        eval time =    2011.41 ms /    18 runs   (  111.74 ms per token,     8.95 tokens per second)
llama_perf_context_print:       total time =    3408.06 ms /    44 tokens


In [102]:
del response['context']

In [105]:
response['definition'].content

'According to the provided text, the context classifies ponds and lakes as "small water bodies."'